# データの前処理について

In [37]:
import pandas as pd
import numpy as np
from io import StringIO

In [8]:
# サンプルデータを作成
csv_data = '''A,B,C,D
                         1.0,2.0,3.0,4.0
                         5.0,6.0,,8.0
                         10.0,11.0,12.0,'''

In [9]:
csv_data

'A,B,C,D\n                         1.0,2.0,3.0,4.0\n                         5.0,6.0,,8.0\n                         10.0,11.0,12.0,'

In [10]:
# サンプルデータを読み込む
df = pd.read_csv(StringIO(csv_data))

In [11]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [12]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [13]:
# DataFrameオブジェクトはNumpyの配列であり、values属性を使っていつでもアクセスできる。
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

## 欠測値を持つサンプル/特徴量を取り除く

In [15]:
# dropnaメソッドは欠測値を含む行を削除したDataFrameを返す。そのためdfに格納されているDataFrameの値は何も変更されていない。
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [17]:
#df

In [21]:
# 欠測値を含む列を削除する。
## axis引数を1に設定すれば、NaNを含んでいる行が1つでもある列を削除できる。
## axis引数を0に設定すれば、列ごとに欠測値を含む行の有無を判定できる。
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [22]:
# dropnaメソッドの他の使用方法
## すべての列がNaNである行だけを削除
df.dropna(how='all')

## 非NaN値が4つ未満の行を削除
df.dropna(thresh=4)

## 特定の列（この場合は'C）にNaNが含まれている行だけを削除
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


## 欠測値を補完する

In [25]:
from sklearn.preprocessing import Imputer

In [26]:
# 欠測値補完のインスタンスを生成(平均値補完)
## axis=0とすると、列ごとの平均値の計算がされる。
## axis=1とすると、行ごとの平均値の計算がされる。
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)

In [27]:
# データを適合
## fitメソッドはトレーニングセットからパラメータを学習するために使用される。
imr = imr.fit(df)

In [28]:
imr

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [29]:
# 補完を実行
## scikit-learnライブラリはnumpyの配列に対応しているため、df.valuesでnumpyの配列にして引数に渡している。
## transformメソッドは学習したパラメータに基づいてデータを変換するために使用される。
imputed_data = imr.transform(df.values)

In [30]:
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [32]:
# imputed_dataとdf.valuesの要素を比べることで、NaN値がそれぞれ特徴量の列ごとの平均値に補完されていることがわかる
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

## データセットをトレーニングデータセットとテストデータセットに分割する

In [34]:
# 178行のワインサンプルとそれらの化学的性質を表す13列の特徴量で構成されているデータセットwineの読み込み
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)

In [36]:
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 
                                   'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

In [38]:
# クラスラベルの表示
print('Class labels', np.unique(df_wine['Class label']))

Class labels [1 2 3]


In [39]:
# wineデータセットの先頭5行を表示
df_wine.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [41]:
# 以下では、train_test_split関数を使用して、トレーニングデータとテストデータにランダムに分割する。
from sklearn.model_selection import train_test_split

In [42]:
# 特徴量とクラスラベルを別々に抽出
## 特徴量の列1〜13のnumpy配列を変数Xに代入し、最初の列（列0）のクラスラベルを変数yに代入する。
### pandasのDataFrameクラスのilocメソッドは、行番号または列番号を指定してデータを抽出する。
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

In [43]:
# トレーニングデータとテストデータに分割
# 全体の30%をテストデータにする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [59]:
X_train

array([[1.371e+01, 1.860e+00, 2.360e+00, ..., 1.110e+00, 4.000e+00,
        1.035e+03],
       [1.222e+01, 1.290e+00, 1.940e+00, ..., 8.600e-01, 3.020e+00,
        3.120e+02],
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       ...,
       [1.242e+01, 1.610e+00, 2.190e+00, ..., 1.060e+00, 2.960e+00,
        3.450e+02],
       [1.390e+01, 1.680e+00, 2.120e+00, ..., 9.100e-01, 3.330e+00,
        9.850e+02],
       [1.416e+01, 2.510e+00, 2.480e+00, ..., 6.200e-01, 1.710e+00,
        6.600e+02]])

## 特徴量の尺度を揃える

### 正規化

In [47]:
from sklearn.preprocessing import MinMaxScaler

In [48]:
# min-maxスケーリングのインスタンスを生成
mms = MinMaxScaler()

In [49]:
# トレーニングデータをスケーリング(正規化(normalization))
X_train_norm = mms.fit_transform(X_train)

In [50]:
# テストデータをスケーリング(正規化(normalization))
X_test_norm = mms.fit_transform(X_test)

In [60]:
X_train_norm

array([[0.72043011, 0.20378151, 0.53763441, ..., 0.48717949, 1.        ,
        0.5854251 ],
       [0.31989247, 0.08403361, 0.31182796, ..., 0.27350427, 0.64102564,
        0.        ],
       [0.60215054, 0.71218487, 0.48387097, ..., 0.04273504, 0.10622711,
        0.42348178],
       ...,
       [0.37365591, 0.1512605 , 0.44623656, ..., 0.44444444, 0.61904762,
        0.02672065],
       [0.77150538, 0.16596639, 0.40860215, ..., 0.31623932, 0.75457875,
        0.54493927],
       [0.84139785, 0.34033613, 0.60215054, ..., 0.06837607, 0.16117216,
        0.28178138]])

### 標準化

In [54]:
from sklearn.preprocessing import StandardScaler

In [55]:
# 標準化のインスタンスを生成（平均=0、標準偏差=1に変換）
stdsc = StandardScaler()

In [56]:
# トレーニングデータをスケーリング(標準化(standardization))
X_train_std = stdsc.fit_transform(X_train)

In [57]:
# テストデータをスケーリング(標準化(standardization))
X_test_std = stdsc.fit_transform(X_test)

In [58]:
X_train_std

array([[ 0.91083058, -0.46259897, -0.01142613, ...,  0.65706596,
         1.94354495,  0.93700997],
       [-0.95609928, -0.96608672, -1.53725357, ..., -0.40859506,
         0.58118003, -1.41336684],
       [ 0.35952243,  1.67501572, -0.37471838, ..., -1.55950896,
        -1.44846566,  0.28683658],
       ...,
       [-0.70550467, -0.68342693, -0.62902295, ...,  0.44393375,
         0.49776993, -1.30608823],
       [ 1.14889546, -0.6215951 , -0.88332752, ..., -0.19546286,
         1.0121322 ,  0.77446662],
       [ 1.47466845,  0.11155374,  0.42452457, ..., -1.43162964,
        -1.23994042, -0.28206514]])